### extend the prgram dependency graph to k-hop inter procedural call

In [1]:
# specify how many hops you want to extend the current graph to
# 0 means no extension
khop = 1

In [2]:
import pickle
import re
from igraph import *

def append_to_graph(g0, g1, conns=None):
    # append a graph to another graph
    # by provided connection: (original node id, new node id)
    g = Graph()
    nv0 = len(g0.vs)
    nv1 = len(g1.vs)
    # add vertices
    g.add_vertices([i for i in range(nv0+nv1)])
    # copy attributes
    for i in range(nv0):
        for dkey in g0.vs[i].attributes().keys():
            g.vs[i][dkey] = g0.vs[i][dkey]
    for i in range(nv1):
        for dkey in g1.vs[i].attributes().keys():
            g.vs[i+nv0][dkey] = g1.vs[i][dkey]
    # add edges
    for e in g0.es:
        g.add_edges([e.tuple])
    for e in g1.es:
        # ascend the index
        g.add_edges([ tuple([e0+nv0 for e0 in e.tuple]) ])
    # add the connection
    if conns is not None:
        for c in conns:
            g.add_edges([ tuple([c[0], c[1]+nv0]) ])
    return g

def find_function_node(g0):
    # return the id of function node of a graph
    ids = []
    for dv in g0.vs:
        if dv["type"] == "Function":
            ids.append(dv.index)
    assert len(ids) == 1
    return ids[0]

def rec_extend_graph(arg_folder, arg_file, arg_k, arg_verbose=False):
    if arg_k <= 0:
        # no need to extend
        return arg_file
    
    file_list = list(arg_folder.keys())
    # print("file_list: {}".format(file_list))
    
    dgraph = arg_file
    d_exgraphs = [] # external graphs that should be connected
    d_conns = [] # connection tuples corresponding to external graphs
    # explore methods called
    for dv in dgraph.vs:
        if dv["type"] == "Function":
            # skip function signature node
            continue
        if "print" in dv["code"]:
            # skip print operation
            continue
        dcalls = re.findall(r"(.*?)\(", dv["code"])
        dcalls = [p.strip() for p in dcalls]
        dcalls = [p for p in dcalls if len(p)>0]
        dfiles = []
        # match method name with file name(s)
        for dc in dcalls:
            mfiles = []
            for df in file_list:
                # if dc in df:
                if dc == df.rsplit("_", 1)[0]:
                    mfiles.append(df)
            # fixme: if there are multiple matches, pick the shortest one
            if len(mfiles)>0:
                if arg_verbose:
                    print("# ({}) matched {} to {}".format(len(mfiles), dc, min(mfiles, key=len)))
                dfiles.append(min(mfiles, key=len))
            else:
                # fixme: can't find matched files, should log out errors/warnings
                if arg_verbose:
                    print("# can't find matching file for method: {}, skip".format(dc))
                pass
        # then append the external graphs and corresponding connections
        for df in dfiles:
            # detect whether the graph is empty or not before adding
            if arg_folder[df].vs is None:
                continue
            if len(arg_folder[df].vs) == 0:
                continue
#             if arg_verbose:
#                 print("# adding external graph: {}".format(df))
            if arg_k == 1:
                # end of k-hop, just append
                d_exgraphs.append(arg_folder[df])
            else:
                # extend k-hop
                kgraph = rec_extend_graph(arg_folder, arg_folder[df], arg_k-1)
                d_exgraphs.append(kgraph)
            # note: the first vertex is not always the function node, so we need to find it out
            d_conns.append((dv.index, find_function_node(arg_folder[df])))
    # then compose the final graph
    ret_graph = dgraph
    for i in range(len(d_exgraphs)):
#         if arg_verbose:
#             print("# merging external graph: {}".format(i))
        ret_graph = append_to_graph(ret_graph, d_exgraphs[i], [d_conns[i]])
    
    return ret_graph

In [49]:
curr_dir = "014"
with open("../SARD_pkl/pdg_db_dir{}.pkl".format(curr_dir), "rb") as f:
    dt = pickle.load(f)

In [50]:
# start extending the graph to k-hop
new_dt = {}
for dfolder in dt.keys():
    # print("# processing folder: {}".format(dfolder))
    print("\r# processing {}/{}".format(len(new_dt.keys()), len(dt.keys())), end="")
    new_dt[dfolder] = {}
    for dfile in dt[dfolder].keys():
        new_dt[dfolder][dfile] = rec_extend_graph(dt[dfolder], dt[dfolder][dfile], khop, arg_verbose=False)

# processing 999/1000

In [51]:
with open("../SARD_pkl/pdg_db_dir{}_{}hop.pkl".format(curr_dir, khop), "wb") as f:
    pickle.dump(new_dt, f)

In [27]:
# gg = rec_extend_graph(dt["120808"], dt["120808"]["main_383036"], 4, arg_verbose=True)

In [109]:
# len(gg.vs)

In [110]:
# for p in gg.vs:
#     p["label_size"]=10
# visual_style = {}
# visual_style["margin"]=40
# visual_style["bbox"]=(600,400)
# visual_style["vertex_label"] = [
#     "{} \n {}".format(gg.vs["name"][i], gg.vs["code"][i]) 
#     for i in range(len(gg.vs["code"]))
# ]
# plot(gg, **visual_style)